In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from functools import reduce
import datetime
from typing import List, Optional 
import pyarrow as pa
import pyarrow.parquet as pq
import sys
sys.path.append("../library/")
import crossexchangearb as cxa

In [2]:
# # Open Binance Futures.
# df_binance_futures = pd.read_parquet('../data/Binance_futures/')
# df_binance_futures = df_binance_futures.assign(exchange_id = 'binance_futures')
# df_binance_futures = df_binance_futures.rename(columns={'timestamp':'timestamp.1'})
# # Open Binance Spot.
# df_binance_spot = pd.read_parquet('../data/Binance_spot/')
# df_binance_spot = df_binance_spot.assign(exchange_id = 'binance_spot')
# df_binance_spot = df_binance_spot.rename(columns={'timestamp':'timestamp.1'})
# Open Binance US Spot.
df_binance_us = pd.read_parquet('../data/Binanceus_spot/')
df_binance_us = df_binance_us.rename(columns={'timestamp':'timestamp.1'})
# Open OKX Futures.
df_okx = pd.read_parquet('../data/OKX_futures/')
df_okx = df_okx.rename(columns={'timestamp':'timestamp.1'})
df_okx.head()

,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
timestamp,,,,,,,,,,,
2022-10-01 15:00:00+00:00,1664636400000,5.3401,5.3401,5.3361,5.3361,189.878030,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2022-10-01 15:01:00+00:00,1664636460000,5.3356,5.3356,5.3262,5.3262,1404.094091,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2022-10-01 15:02:00+00:00,1664636520000,5.3273,5.3318,5.3273,5.3313,1525.072300,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2022-10-01 15:03:00+00:00,1664636580000,5.3316,5.3379,5.3316,5.3354,530.874701,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2022-10-01 15:04:00+00:00,1664636640000,5.3348,5.3348,5.3303,5.3303,259.569769,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10


In [3]:
cxa.postprocess_exchange_data(df_okx).head()

close                                       \
                               okx                                        
                          APE_USDT AVAX_USDT AXS_USDT BNB_USDT BTC_USDT   
timestamp                                                                 
2021-01-01 15:00:00+00:00      NaN     3.728      NaN      NaN  29329.3   
2021-01-01 15:01:00+00:00      NaN     3.728      NaN      NaN  29292.2   
2021-01-01 15:02:00+00:00      NaN     3.726      NaN      NaN  29259.0   
2021-01-01 15:03:00+00:00      NaN     3.719      NaN      NaN  29291.8   
2021-01-01 15:04:00+00:00      NaN     3.719      NaN      NaN  29269.2   

                                                                         ...  \
                                                                         ...   
                          DOGE_USDT DOT_USDT ETH_USDT FTM_USDT GMT_USDT  ...   
timestamp                                                                ...   
2021-01-01 15:00:00+00:00  0.004949    8.666   738.21   0.0173      NaN  ...   
2021-01-01 15:01:00+00:00  0.004938    8.673   737.80   0.0174      NaN  ...   
2021-01-01 15:02:00+00:00  0.004953    8.667   736.80   0.0174      NaN  ...   
2021-01-01 15:03:00+00:00  0.004950    8.678   736.54   0.0175      NaN  ...   
2021-01-01 15:04:00+00:00  0.004952    8.673   736.10   0.0175      NaN  ...   

                               vwap                                           \
                                okx                                            
                           FTM_USDT GMT_USDT  LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                                      
2021-01-01 15:00:00+00:00  0.017400      NaN  12.136000        NaN  1.365400   
2021-01-01 15:01:00+00:00  0.017420      NaN  12.147935        NaN  1.365355   
2021-01-01 15:02:00+00:00  0.017413      NaN  12.149790        NaN  1.365157   
2021-01-01 15:03:00+00:00  0.017410      NaN  12.153976        NaN  1.365119   
2021-01-01 15:04:00+00:00  0.017408      NaN  12.159146        NaN  1.365020   

                                                                               
                                                                               
                          SAND_USDT  SOL_USDT STORJ_USDT WAVES_USDT  XRP_USDT  
timestamp                                                                      
2021-01-01 15:00:00+00:00       NaN  1.642000        NaN   6.225000  0.237200  
2021-01-01 15:01:00+00:00       NaN  1.641388     0.2992   6.223844  0.237389  
2021-01-01 15:02:00+00:00       NaN  1.638708     0.2992   6.222253  0.237936  
2021-01-01 15:03:00+00:00       NaN  1.637566     0.2992   6.218422  0.238043  
2021-01-01 15:04:00+00:00       NaN  1.637444     0.2992   6.216047  0.238044  

[5 rows x 108 columns]

In [4]:
df = cxa.merge_postprocess_exchange_data([df_okx, df_binance_us])
df.head()

close                                        \
                          binanceus                                         
                           APE_USDT AVAX_USDT AXS_USDT BNB_USDT  BTC_BUSD   
timestamp                                                                   
2021-01-01 00:00:00+00:00       NaN       NaN      NaN  37.3526  28968.28   
2021-01-01 00:01:00+00:00       NaN       NaN      NaN  37.3711  29061.51   
2021-01-01 00:02:00+00:00       NaN       NaN      NaN  37.3402  29029.86   
2021-01-01 00:03:00+00:00       NaN       NaN      NaN  37.3402  28999.86   
2021-01-01 00:04:00+00:00       NaN       NaN      NaN  37.3402  29017.34   

                                                                          ...  \
                                                                          ...   
                           BTC_USDT CRV_USDT DOGE_USDT DOT_USDT ETH_BUSD  ...   
timestamp                                                                 ...   
2021-01-01 00:00:00+00:00  28951.91      NaN  0.004688      NaN   737.00  ...   
2021-01-01 00:01:00+00:00  29014.36      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:02:00+00:00  28995.55      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:03:00+00:00  28974.27      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:04:00+00:00  28979.21      NaN  0.004688      NaN   738.03  ...   

                              vwap                                          \
                               okx                                           
                          FTM_USDT GMT_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                                    
2021-01-01 00:00:00+00:00      NaN      NaN       NaN        NaN       NaN   
2021-01-01 00:01:00+00:00      NaN      NaN       NaN        NaN       NaN   
2021-01-01 00:02:00+00:00      NaN      NaN       NaN        NaN       NaN   
2021-01-01 00:03:00+00:00      NaN      NaN       NaN        NaN       NaN   
2021-01-01 00:04:00+00:00      NaN      NaN       NaN        NaN       NaN   

                                                                             
                                                                             
                          SAND_USDT SOL_USDT STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                                    
2021-01-01 00:00:00+00:00       NaN      NaN        NaN        NaN      NaN  
2021-01-01 00:01:00+00:00       NaN      NaN        NaN        NaN      NaN  
2021-01-01 00:02:00+00:00       NaN      NaN        NaN        NaN      NaN  
2021-01-01 00:03:00+00:00       NaN      NaN        NaN        NaN      NaN  
2021-01-01 00:04:00+00:00       NaN      NaN        NaN        NaN      NaN  

[5 rows x 228 columns]

In [9]:
cxa.get_coins(df)

['APE_USDT',
 'AVAX_USDT',
 'AXS_USDT',
 'BNB_USDT',
 'BTC_BUSD',
 'BTC_USDT',
 'CRV_USDT',
 'DOGE_USDT',
 'DOT_USDT',
 'ETH_BUSD',
 'ETH_USDT',
 'FTM_USDT',
 'GMT_USDT',
 'LINK_USDT',
 'MATIC_USDT',
 'NEAR_USDT',
 'OGN_USDT',
 'SAND_USDT',
 'SOL_USDT',
 'STORJ_USDT',
 'WAVES_USDT',
 'XRP_USDT']

In [10]:
cxa.get_ncoins(df)

22

In [13]:
cxa.get_coins_info(df, 'ETH_USDT').head()

binanceus                                    \
                              close    high     low    open    volume   
timestamp                                                               
2021-01-01 00:00:00+00:00    736.24  736.24  736.24  736.24   0.00000   
2021-01-01 00:01:00+00:00    739.07  739.07  737.61  737.61  10.53080   
2021-01-01 00:02:00+00:00    738.46  738.46  738.46  738.46   1.41407   
2021-01-01 00:03:00+00:00    737.74  737.74  737.74  737.74   0.40661   
2021-01-01 00:04:00+00:00    737.90  737.90  737.90  737.90   0.03389   

                                        okx                            
                                 vwap close high low open volume vwap  
timestamp                                                              
2021-01-01 00:00:00+00:00         NaN   NaN  NaN NaN  NaN    NaN  NaN  
2021-01-01 00:01:00+00:00  738.340000   NaN  NaN NaN  NaN    NaN  NaN  
2021-01-01 00:02:00+00:00  738.354206   NaN  NaN NaN  NaN    NaN  NaN  
2021-01-01 00:03:00+00:00  738.333986   NaN  NaN NaN  NaN    NaN  NaN  
2021-01-01 00:04:00+00:00  738.332799   NaN  NaN NaN  NaN    NaN  NaN

In [7]:
cxa.get_exchanges(df)

['binanceus', 'okx']

In [8]:
cxa.get_features(df)

['close', 'high', 'low', 'open', 'volume', 'vwap']

In [12]:
cxa.get_exchange_info(df, 'binanceus').head()

close                                        \
                          APE_USDT AVAX_USDT AXS_USDT BNB_USDT  BTC_BUSD   
timestamp                                                                  
2021-01-01 00:00:00+00:00      NaN       NaN      NaN  37.3526  28968.28   
2021-01-01 00:01:00+00:00      NaN       NaN      NaN  37.3711  29061.51   
2021-01-01 00:02:00+00:00      NaN       NaN      NaN  37.3402  29029.86   
2021-01-01 00:03:00+00:00      NaN       NaN      NaN  37.3402  28999.86   
2021-01-01 00:04:00+00:00      NaN       NaN      NaN  37.3402  29017.34   

                                                                          ...  \
                           BTC_USDT CRV_USDT DOGE_USDT DOT_USDT ETH_BUSD  ...   
timestamp                                                                 ...   
2021-01-01 00:00:00+00:00  28951.91      NaN  0.004688      NaN   737.00  ...   
2021-01-01 00:01:00+00:00  29014.36      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:02:00+00:00  28995.55      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:03:00+00:00  28974.27      NaN  0.004688      NaN   738.78  ...   
2021-01-01 00:04:00+00:00  28979.21      NaN  0.004688      NaN   738.03  ...   

                                 vwap                                          \
                             ETH_USDT FTM_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                                       
2021-01-01 00:00:00+00:00         NaN      NaN       NaN        NaN       NaN   
2021-01-01 00:01:00+00:00  738.340000      NaN       NaN        NaN       NaN   
2021-01-01 00:02:00+00:00  738.354206      NaN       NaN        NaN       NaN   
2021-01-01 00:03:00+00:00  738.333986      NaN       NaN        NaN       NaN   
2021-01-01 00:04:00+00:00  738.332799      NaN       NaN        NaN       NaN   

                                                                            
                          OGN_USDT SAND_USDT SOL_USDT STORJ_USDT  XRP_USDT  
timestamp                                                                   
2021-01-01 00:00:00+00:00      NaN       NaN      NaN        NaN  0.219180  
2021-01-01 00:01:00+00:00      NaN       NaN      NaN        NaN  0.220009  
2021-01-01 00:02:00+00:00      NaN       NaN      NaN        NaN  0.220009  
2021-01-01 00:03:00+00:00      NaN       NaN      NaN        NaN  0.220009  
2021-01-01 00:04:00+00:00      NaN       NaN      NaN        NaN  0.220009  

[5 rows x 120 columns]

In [14]:
cxa.get_feature_info(df, 'vwap').head()

binanceus                                \
                           APE_USDT AVAX_USDT AXS_USDT   BNB_USDT   
timestamp                                                           
2021-01-01 00:00:00+00:00       NaN       NaN      NaN  37.352600   
2021-01-01 00:01:00+00:00       NaN       NaN      NaN  37.371613   
2021-01-01 00:02:00+00:00       NaN       NaN      NaN  37.362572   
2021-01-01 00:03:00+00:00       NaN       NaN      NaN  37.362572   
2021-01-01 00:04:00+00:00       NaN       NaN      NaN  37.362572   

                                                                          \
                               BTC_BUSD      BTC_USDT CRV_USDT DOGE_USDT   
timestamp                                                                  
2021-01-01 00:00:00+00:00  28967.490000  28943.335000      NaN       NaN   
2021-01-01 00:01:00+00:00  29034.744841  28986.477367      NaN       NaN   
2021-01-01 00:02:00+00:00  29035.266318  28986.958822      NaN       NaN   
2021-01-01 00:03:00+00:00  29032.653630  28986.846904      NaN       NaN   
2021-01-01 00:04:00+00:00  29031.959243  28986.856765      NaN       NaN   

                                                ...      okx           \
                          DOT_USDT    ETH_BUSD  ... FTM_USDT GMT_USDT   
timestamp                                       ...                     
2021-01-01 00:00:00+00:00      NaN         NaN  ...      NaN      NaN   
2021-01-01 00:01:00+00:00      NaN  738.780000  ...      NaN      NaN   
2021-01-01 00:02:00+00:00      NaN  738.780000  ...      NaN      NaN   
2021-01-01 00:03:00+00:00      NaN  738.780000  ...      NaN      NaN   
2021-01-01 00:04:00+00:00      NaN  738.664077  ...      NaN      NaN   

                                                                             \
                          LINK_USDT MATIC_USDT NEAR_USDT SAND_USDT SOL_USDT   
timestamp                                                                     
2021-01-01 00:00:00+00:00       NaN        NaN       NaN       NaN      NaN   
2021-01-01 00:01:00+00:00       NaN        NaN       NaN       NaN      NaN   
2021-01-01 00:02:00+00:00       NaN        NaN       NaN       NaN      NaN   
2021-01-01 00:03:00+00:00       NaN        NaN       NaN       NaN      NaN   
2021-01-01 00:04:00+00:00       NaN        NaN       NaN       NaN      NaN   

                                                          
                          STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                 
2021-01-01 00:00:00+00:00        NaN        NaN      NaN  
2021-01-01 00:01:00+00:00        NaN        NaN      NaN  
2021-01-01 00:02:00+00:00        NaN        NaN      NaN  
2021-01-01 00:03:00+00:00        NaN        NaN      NaN  
2021-01-01 00:04:00+00:00        NaN        NaN      NaN  

[5 rows x 38 columns]